In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
# 모델, 텐서 연산
import torch.nn as nn
import torch.optim as optim

from tqdm.notebook import tqdm_notebook
# 학습 진행상황 표시용

In [ ]:
from google.colab import drive
# Google Drive에서 데이터 불러오기
drive.mount('/content/drive')
# Google Drive를 Colab에 마운트해서 파일 접근 가능하게 함

In [ ]:
def show(img): ##visualize your image
    plt.imshow(img[1:].reshape(28, 28), cmap='gray')
    plt.show()
    # 1번째 값(레이블)을 제외한 나머지를 28x28로 reshape해서 흑백 이미지로 출력하는 함수

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/mnist_train.csv')
pxl = ["pxl{}".format(i) for i in range(len(df.columns)-1)]
df.columns = ["value"] + pxl
#df.rename(columns = {"5":"value"}, inplace = True)
# MNIST 학습 데이터를 CSV 파일에서 읽고, 컬럼명 정리

In [ ]:
df.head(10)

In [ ]:
show(df.iloc[3].values) #visualize 3rd dataset (hand writing image)
# 데이터 일부 확인 및 4번째 손글씨 이미지 출력

In [ ]:
A = pd.DataFrame(df, columns = pxl).values
y_data = pd.DataFrame(df, columns = ["value"]).values
# 입력 이미지 (A)와 정답 레이블 (y_data) 분리

In [ ]:
A.shape

In [ ]:
def digit_to_vec(value):
  vec = np.zeros(shape = 10)
  vec[value] = 1
  return vec
# 정답 숫자(label)를 0~9 원-핫 벡터로 변환

In [ ]:
y_data_vec = [digit_to_vec(y_data[k]) for k in range(len(y_data))]
B = np.array(y_data_vec)
# 전체 정답 레이블을 원-핫 인코딩된 배열로 변환 → B

In [ ]:
B

In [ ]:
x = torch.tensor(A, dtype=torch.float32)
y = torch.tensor(B, dtype=torch.float32)
x = x.unsqueeze(1) #new dimension at 1st comp.
y = y.unsqueeze(1) #new dimension at 1st comp.
# 데이터를 float32 타입 PyTorch 텐서로 변환
# CNN 입력을 위해 (batch_size, 1, 784) 형태로 차원 추가

In [ ]:
x.shape

In [ ]:
x[0].shape

In [ ]:
conv = nn.Conv1d(in_channels=1, out_channels=2, kernel_size=3)

In [ ]:
list(conv.parameters())

In [ ]:
conv(x[0]).shape
# 간단한 Conv1d 레이어 정의 후 x[0]에 적용해 출력 shape 확인

In [ ]:
conv = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3)

In [ ]:
x1 = conv(x[0])
# 채널 수 1인 Convolution 적용

In [ ]:
print(x1.shape)

In [ ]:
pool = nn.MaxPool1d(kernel_size=2)

In [ ]:
list(pool.parameters())

In [ ]:
pool(x1).shape
# Max Pooling을 적용해 차원 줄임

In [ ]:
x2 = pool(x1)
print(x2.shape)

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=1, out_channels=1, kernel_size=3)

        self.linear = nn.Linear(96,10)
        self.pool = nn.AvgPool1d(kernel_size=2)

        self.softmax = nn.Softmax(dim=2)

    def forward(self, x):
        #x = self.dropout(x)  # Dropout for input layer
        x = self.conv1(x)
        x = self.pool(x)
        #print(x.shape)

        x = self.conv2(x)  # Dropout after first ReLU
        x = self.pool(x)
        #print(x.shape)

        x = self.conv3(x)
        x = self.pool(x)
        #print(x.shape)

        x = self.linear(x)
        x = self.softmax(x)
        return x
    # Conv1d → Pool → Conv1d → Pool → Conv1d → Pool → Linear → Softmax 구조의 CNN 정의

In [ ]:
model = CNN()

In [ ]:
model(x[:2])
# CNN 모델 객체 생성 후 x[:2]에 대한 예측 실행

In [ ]:
x.shape

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-2)

loss_fn = nn.MSELoss()

num_epochs = 1000
# Adam 옵티마이저, MSE Loss, 1000 에폭 설정

In [ ]:
for epoch in tqdm_notebook(range(num_epochs)):
    model.train()

    # Forward
    output = model(x)
    loss = loss_fn(output, y)

    # Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if epoch % 5 == 0:
      print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {loss.item():.4f}")
      # 에폭마다 모델 학습 수행 및 손실 출력

In [ ]:
model = CNN()  # 동일한 구조의 모델 객체 생성
model.load_state_dict(torch.load('model.pth'))
model.eval()
# 저장된 모델 파라미터 불러오고 평가모드로 전환

In [ ]:
df2=pd.read_csv('/content/drive/MyDrive/mnist_test.csv')
pxl = ["pxl{}".format(i) for i in range(len(df.columns)-1)]
df2.columns = ["value"] + pxl
# 테스트 데이터 CSV 읽고 컬럼 설정

A2 = pd.DataFrame(df2, columns = pxl).values
x_test = torch.tensor(A2, dtype=torch.float32).unsqueeze(1)
# 입력 데이터 텐서로 변환하고 차원 추가

y_data2 = pd.DataFrame(df2, columns = ["value"]).values
y_data2 = y_data2.reshape(-1)
# 테스트 레이블 추출 및 1차원으로 변형

In [ ]:
model.eval()
y_pred = model(x_test).squeeze(1).detach().cpu().numpy()
y_pred = np.argmax(y_pred,axis=1)
# 모델 예측 결과를 NumPy로 변환하고 softmax 결과에서 가장 큰 인덱스를 예측 숫자로 사용

In [ ]:
y_pred

In [ ]:
y_data2

In [ ]:
(y_pred == y_data2).sum() / len(y_data2)
# 예측값과 실제값 비교해 정확도 계산